In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

%matplotlib inline
plt.rcParams['font.family'] = 'Hiragino sans'
plt.style.use("ggplot")

In [4]:
# 使用するcsvを読み込む
df_dex = pd.read_csv('DEXJPUS.csv') # 年別のドル円為替相場
df_rate = pd.read_csv('gb_rate.csv') # 年別のアメリカと日本の国債利回り
df_M1 = pd.read_csv('M1.csv') # 年別のアメリカと日本のM1（現金通貨＋預金通貨）
df_rGDP = pd.read_csv('REAL_GDP.csv') # 年別のアメリカと日本の実質GDP

## データを確認する

In [5]:
df_dex.columns = ['DATE','為替']
df_dex.head()

,DATE,為替
0,1971-01-01,347.78569
1,1972-01-01,303.12498
2,1973-01-01,271.30548
3,1974-01-01,291.84460
4,1975-01-01,296.78488


In [6]:
df_rate.columns = ['DATE','US_国債利回り','JP_国債利回り']
df_rate.head()

,DATE,US_国債利回り,JP_国債利回り
0,1960-01-01,4.1166666666666667,.
1,1961-01-01,3.8825000000000000,.
2,1962-01-01,3.9458333333333333,.
3,1963-01-01,4.0025000000000000,.
4,1964-01-01,4.1866666666666667,.


In [7]:
df_M1.columns = ['DATE', 'JP_M1', 'US_M1']
df_M1.head()

,DATE,JP_M1,US_M1
0,1959-01-01,.,140.4166666666666667
1,1960-01-01,4350533034083.7326,140.3083333333333333
2,1961-01-01,5443836847605.4202,143.0500000000000000
3,1962-01-01,6312149686198.0741,146.4916666666666667
4,1963-01-01,7460868868406.0380,150.9583333333333333


In [8]:
df_rGDP.columns = ['DATE','US_rGDP','JP_rGDP']
df_rGDP.head()

,DATE,US_rGDP,JP_rGDP
0,1947-01-01,2036.2040000000000000,.
1,1948-01-01,2119.9592500000000000,.
2,1949-01-01,2107.9990000000000000,.
3,1950-01-01,2291.1105000000000000,.
4,1951-01-01,2475.4487500000000000,.


## これらを１つのdfにマージする

In [9]:
tmp = df_rate.merge(df_M1)
tmp = tmp.merge(df_rGDP)
tmp = tmp.merge(df_dex)

In [10]:
# 1993年以降でないと全てのデータがないため、本分析では1993年以降のデータを使用することとする
df_test = tmp[tmp['DATE']>'1993-01-01']
df_test

,DATE,US_国債利回り,JP_国債利回り,JP_M1,US_M1,US_rGDP,JP_rGDP,為替
23,1994-01-01,7.0800000000000000,4.3633333333333333,165930005760708.3333,1145.2166666666666667,10358.9232500000000000,446538.800000000000,102.178964
24,1995-01-01,6.5800000000000000,3.4435000000000000,179475682407083.6667,1143.0083333333333333,10636.9787500000000000,458237.550000000000,93.964940
25,1996-01-01,6.4383333333333333,3.1015000000000000,204092412385529.6667,1106.8166666666666667,11038.2660000000000000,472690.700000000000,108.780000
26,1997-01-01,6.3525000000000000,2.3736666666666667,221886430305457.3258,1070.2333333333333333,11529.1570000000000000,477476.150000000000,121.058127
27,1998-01-01,5.2641666666666667,1.5414166666666667,240855013708597.5858,1080.6166666666666667,12045.8240000000000000,471182.400000000000,130.989167
28,1999-01-01,5.6366666666666667,1.7490000000000000,266201438044158.5025,1102.3000000000000000,12623.361000000000,470023.425000000000,113.734246
29,2000-01-01,6.0291666666666667,1.7444166666666667,287978996714942.6667,1103.7000000000000000,13138.035250000000,482804.850000000000,107.804048
30,2001-01-01,5.0175000000000000,1.3190000000000000,312504848413047.4167,1140.2416666666666667,13263.417000000000,484729.400000000000,121.568040
31,2002-01-01,4.6108333333333333,1.2631666666666667,398448057227801.0050,1196.6833333333333333,13488.357000000000,484755.475000000000,125.220438
32,2003-01-01,4.0150000000000000,1.00325000000000000000,430543829786152.4117,1274.0666666666666667,13865.519250000000,492387.775000000000,115.938685


## object型をfloat型に変換する

In [11]:
# dataを数値に変換したい。
# もともとはobjectになってる
df_test.dtypes

DATE         object
US_国債利回り     object
JP_国債利回り     object
JP_M1        object
US_M1        object
US_rGDP      object
JP_rGDP      object
為替          float64
dtype: object

In [12]:
dict_types = {'US_国債利回り':'float64', 'JP_国債利回り':'float64', 'JP_M1':'float64', 'US_M1':'float64', 'US_rGDP':'float64', 'JP_rGDP':'float64'}

df_test = df_test.astype(dict_types)
df_test.dtypes

DATE         object
US_国債利回り    float64
JP_国債利回り    float64
JP_M1       float64
US_M1       float64
US_rGDP     float64
JP_rGDP     float64
為替          float64
dtype: object

## 重回帰分析を行う

In [13]:
import statsmodels.api as sm

y = df_test['為替']
x = df_test[['US_国債利回り', 'JP_国債利回り', 'JP_M1', 'US_M1', 'US_rGDP', 'JP_rGDP']]

X = sm.add_constant(x)

/Users/gr39my/.pyenv/versions/3.8.5/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [14]:
model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     為替   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     1.554
Date:                Wed, 09 Feb 2022   Prob (F-statistic):              0.228
Time:                        20:11:53   Log-Likelihood:                -104.00
No. Observations:                  27   AIC:                             216.0
Df Residuals:                      23   BIC:                             221.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.013     -0.026      0.980      -0.028       0.028
US_国債利回り      -0.0919      3.586     -0.026      0.980      -7.510       7.327
JP_国債利回り      -0.0545      2.125     -0.026      0.980      -4.450       4.341
JP_M1       7.773e-14    8.8e-14      0.883      0.386   -1.04e-13     2.6e-13
US_M1         -0.0003      0.002     -0.133      0.895      -0.004       0.004
US_rGDP       -0.0126      0.009     -1.347      0.191      -0.032       0.007
JP_rGDP        0.0005      0.000      2.111      0.046    1.02e-05       0.001
==============================================================================
Omnibus:                        0.492   Durbin-Watson:                   0.680
Prob(Omnibus):                  0.782   Jarque-Bera (JB):                0.440
Skew:                          -0.277   Prob(JB):                        0.802
Kurtosis:                       2.711   Cond. No.                     2.81e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.81e+16. This might indicate that there are
strong multicollinearity or other numerical problems.
"""